# KAZNIVA DEJANJA V SLOVENIJI

## 1. Predstavitev in predprocesiranje podatkov

In [3]:
import pandas as pd

df = pd.read_csv("data/kd2019.csv", delimiter=";")
print(df["DanVTednu"])

0         SOBOTA
1          PETEK
2          PETEK
3          SREDA
4          SREDA
           ...  
105340     SREDA
105341     SREDA
105342     SREDA
105343     PETEK
105344     PETEK
Name: DanVTednu, Length: 105345, dtype: object


## 2. Osnovna vizualizacija